In [1]:
import requests 
from bs4 import BeautifulSoup
import re 
import collections 
import json 

In [2]:
def HeaderStr2HeadersDict(headerStr):
    """
    转换F12、等工具获得的请求头字符串成为python字典形式 
    """
    a1 = headerStr.split('\n')[1:-1]
    headers = {}
    for parmer in a1:
        st1 = parmer.split(':')
        key = st1[0]
        value = st1[1]    
        headers[key] =value
    return headers

In [198]:
class ScholarArticle(object):
    """
    A class representing articles listed on Google Scholar.  The class
    provides basic dictionary-like behavior.
    """
    def __init__(self):
        # The triplets for each keyword correspond to (1) the actual
        # value, (2) a user-suitable label for the item, and (3) an
        # ordering index:
        self.attrs = {
            'title':         [None, 'Title',          0],
            'url':           [None, 'URL',            1],
            'year':          [None, 'Year',           2],
            'num_citations': [0,    'Citations',       3],
            'infor':         [0,    'information',     4],
            'url_pdf':       [None, 'PDF link',        5],
            'url_citations': [None, 'Citations list',  6],
            'url_citation':  [None, 'Citation link',   7],
        }

        # The citation data in one of the standard export formats,
        # e.g. BibTeX.
        self.citation_data = None

    def __getitem__(self, key):
        if key in self.attrs:
            return self.attrs[key][0]
        return None

    def __len__(self):
        return len(self.attrs)

    def __setitem__(self, key, item):
        if key in self.attrs:
            self.attrs[key][0] = item
        else:
            self.attrs[key] = [item, key, len(self.attrs)]

    def __delitem__(self, key):
        if key in self.attrs:
            del self.attrs[key]
    
    def set_citation_data(self,citation_data):
        self.citation_data = citation_data
    
    

In [241]:
class googleScholarSpider(object):
    def __init__(self,key,maxPage=30):
        self.key = key
        self.host = 'https://scholar.google.com'
        self.result = []
        self.start = 0
        self.url = 'https://scholar.google.com/scholar?start={}&q={}&hl=zh-CN&as_sdt=1,5&as_vis=1'
        self.maxPage = maxPage
        self.header = {'accept-encoding': 'gzip, deflate, br',
 'accept-language': 'zh-CN,zh;q=0.9',
 'cookie': 'HSID=AKuiss57lEl054xJI; SSID=AtICnLzEUSHXkEave; APISID=x9mwoA3nAdcZA2YV/A_bzkaWhuB7L5muFB; SAPISID=eWDJEPb7Jvp9GbpC/AVFVmMA3M2VH4UShY; __Secure-HSID=AKuiss57lEl054xJI; __Secure-SSID=AtICnLzEUSHXkEave; __Secure-APISID=x9mwoA3nAdcZA2YV/A_bzkaWhuB7L5muFB; __Secure-3PAPISID=eWDJEPb7Jvp9GbpC/AVFVmMA3M2VH4UShY; CONSENT=YES+US.zh-CN+201907; GSP=LM=1581249166',
 'referer': 'https',
 'sec-fetch-dest': 'document',
 'sec-fetch-mode': 'navigate',
 'sec-fetch-site': 'same-origin',
 'sec-fetch-user': '?1',
 'upgrade-insecure-requests':'1',
 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.113 Safari/537.36'}
    
    
    def downloadPage(self):
        
        for num in range(self.maxPage):
            start = num*20
            url = self.url.format(start,self.key)
            r = requests.get(url,headers = self.header)
            soup = BeautifulSoup(r.text,'html.parser')
            listPaper = soup.find('div',id='gs_res_ccl_mid').find_all('div',class_='gs_r gs_or gs_scl')
            for each in listPaper:
                article = ScholarArticle()
                
                article['url_pdf'] =each.find('div',class_='gs_ggs gs_fl').find('a')['href']
                article['url'] = each.find('div',class_='gs_ri').find('h3').find('a')['href']
                article['title'] = each.find('div',class_='gs_ri').find('h3').text
                article['infor'] =  each.find('div',class_='gs_ri').find('div',class_ ='gs_a').text
            
                article['num_citations'] = each.find('div',class_='gs_ri').find('a',text=re.compile('被引用次数')).text 
                article['url_citation'] = self.host +each.find('div',class_='gs_ri').find('a',text=re.compile('被引用次数'))['href']                
                
                self.result.append(article)
                break
            return soup 
            break 
       
            

        
        

In [239]:
a = googleScholarSpider(key = 'SARS-Cov-2')

In [240]:
s = a.downloadPage()

In [237]:
listPaper = s.find('div',id='gs_res_ccl_mid').find_all('div',class_='gs_r gs_or gs_scl')
for each in listPaper:
    h = each.find('div',class_='gs_ri').find('a',text=re.compile('被引用次数'))['href']
    print(h)
    break

/scholar?cites=12709711274646573724&as_sdt=2005&sciodt=1,5&hl=zh-CN


In [245]:
a.result[0].attrs['url_citation'][0]

'https://scholar.google.com//scholar?cites=12709711274646573724&as_sdt=2005&sciodt=1,5&hl=zh-CN'